In [1]:
import matplotlib.pyplot as plt


In [129]:
import os
import numpy as np
from PIL import Image
import json
import cv2
from sklearn.cluster import MiniBatchKMeans

location = "D:\\Training_Data_SVM_Control\\"


imageList = []
labelList = []
ytrain = []

for dirName, x, fileList in os.walk(location):
    
    for filename in fileList:
        if(filename[-4:] == ".jpg" and not filename[-7:] == "ged.jpg"):
            
            json_file_location = os.path.join(dirName, filename[:-4] + ".json")
            
            if os.path.exists(json_file_location):
                
                jpg_file_location = os.path.join(dirName, filename)
                imageList.append(str(jpg_file_location))
                labelList.append(str(json_file_location))

for json_file in labelList:
    file = open(json_file)
    data = json.load(file)
    ytrain.append(data["Pano_Data"][0]["label_type"])

    
    
X = np.array([np.array(Image.open(fname)) for fname in imageList])    
y = np.array(ytrain,)

y = y.reshape(y.shape[0],)
image_features = X.reshape(X.shape[1] * X.shape[2] * X.shape[3], X.shape[0]).T

In [125]:
#Getting null Data too

null_location = "D:\\Training_Data_SVM_null\\"
null_img_list = []
null_y_train = []
for dirName, x, fileList in os.walk(null_location):
    for filename in fileList:
        if(filename[-4:] == ".jpg" and not filename[-7:] == "ged.jpg" and not filename[0] == '.'):
            null_jpg_location = os.path.join(dirName, filename)
            null_img_list.append(null_jpg_location)
            null_y_train.append("null")
            
null_y_train = np.array(null_y_train).reshape(len(null_y_train),)

null_img_list = np.array([np.array(Image.open(fname)) for fname in null_img_list])
null_img_list = null_img_list.reshape(null_img_list.shape[1] * null_img_list.shape[2] * null_img_list.shape[3], null_img_list.shape[0]).T


In [137]:
new_feature_set = np.concatenate((image_features, null_img_list) , 0)
new_y = np.concatenate((y, null_y_train), 0)
new_feature_set.shape


(2511, 181548)

In [7]:
def get_sift_dict(image_list, n_classes, sift):
    sift_descriptors = []
    
    
    for image_path in image_list:
        img = cv2.imread(image_path)
        kp, des = sift.detectAndCompute(img, None)
        print(image_path, end = '\r')
        for d in des:
            sift_descriptors.append(d)
    k = 10 * n_classes
    batch_size = np.size(len(image_list)) * 3
    
    kmeans = MiniBatchKMeans(n_clusters = k, batch_size = batch_size, verbose = 1).fit(sift_descriptors)
    kmeans.verbose = False
    
    return kmeans

def get_sift_features(n_classes,image_list, feature_dict, sift):
    histo_list = []

    for img_path in image_list:
        img = cv2.imread(img_path)
        kp, des = sift.detectAndCompute(img, None)
        k = 10 * n_classes
        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = feature_dict.predict([d])
            histo[idx] += 1/nkp

        histo_list.append(histo)
    return np.array(histo_list)

In [50]:
sift = cv2.xfeatures2d.SIFT_create()

sift_dict = get_sift_dict(imageList, 50, sift)




Init 1/3 with method: k-means++stacle\8q\8qql7NbVqcC_EnaW43WeCw_._Obstacle_._2842.890758547011_._1228.923076923077_._.jpggg.jpg_._.jpgg
Inertia for init 1/3: 0.000000
Init 2/3 with method: k-means++


C:\Users\ubicomp\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:1418: RuntimeWarning: init_size=9 should be larger than k=500. Setting it to 3*k
  init_size=init_size)


Inertia for init 2/3: 0.000000
Init 3/3 with method: k-means++
Inertia for init 3/3: 0.000000
Minibatch iteration 1/22320500: mean batch inertia: 87096.333333, ewa inertia: 87096.333333 
Minibatch iteration 2/22320500: mean batch inertia: 92712.333333, ewa inertia: 87096.383655 
Minibatch iteration 3/22320500: mean batch inertia: 88152.666667, ewa inertia: 87096.393119 
Minibatch iteration 4/22320500: mean batch inertia: 103660.333333, ewa inertia: 87096.541538 
Minibatch iteration 5/22320500: mean batch inertia: 122824.666667, ewa inertia: 87096.861675 
Minibatch iteration 6/22320500: mean batch inertia: 63560.000000, ewa inertia: 87096.650776 
Minibatch iteration 7/22320500: mean batch inertia: 106429.333333, ewa inertia: 87096.824004 
Minibatch iteration 8/22320500: mean batch inertia: 41229.666667, ewa inertia: 87096.413018 
Minibatch iteration 9/22320500: mean batch inertia: 118722.000000, ewa inertia: 87096.696395 
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch iterat

In [51]:
sift_features = get_sift_features(50,imageList, sift_dict, sift)

In [52]:
sift_features.shape


(1998, 500)

In [53]:
from sklearn.utils import shuffle

X = sift_features
y = y.reshape(y.shape[0],)

X, y = shuffle(X, y, random_state=82)

In [62]:
from sklearn.model_selection import train_test_split
from sklearn import svm 
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(image_features, y, test_size = 0.05, random_state = 400)

clf = RandomForestClassifier()

clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [63]:
from sklearn.metrics import accuracy_score, confusion_matrix


preds = clf.predict(X_test)

print(confusion_matrix(y_test, preds))
print("Accurancy:", accuracy_score(y_test, preds))

[[25  0  0  0]
 [ 0 22  0  0]
 [ 0  0 24  0]
 [ 0  0  0 29]]
Accurancy: 1.0


In [35]:
print(len(X_test))

400


In [48]:
import sklearn
print(sklearn.metrics.confusion_matrix(y_test, preds))
print(sklearn.metrics.accuracy_score(y_test ,preds))

[[239   0   0   0]
 [276   0   0   0]
 [245   0   0   0]
 [239   0   0   0]]
0.23923923923923923


In [46]:
(y_train == "SurfaceProblem").sum()

258

In [138]:
def data_analysis(clf, X_test, y_test):
    preds = clf.predict(X_test)

    print(confusion_matrix(y_test, preds))
    print("Accurancy:", accuracy_score(y_test, preds))

In [139]:
from sklearn.model_selection import KFold

number_of_folds = 5
randomize = True

kfold = KFold(number_of_folds, randomize, 4)

for train, test in kfold.split(new_feature_set):
    X_train = new_feature_set[train]
    X_test = new_feature_set[test]
    
    y_train = new_y[train]
    y_test = new_y[test]
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    data_analysis(clf, X_test, y_test)

[[113   0   0   0   0]
 [  0  99   0   0   0]
 [  0   0  84   0   0]
 [  0   0   0  96   0]
 [  0   0   0   0 111]]
Accurancy: 1.0
[[ 96   0   0   0   0]
 [  0 100   0   0   0]
 [  0   1  91   0   0]
 [  0   0   0  99   0]
 [  0   0   0   0 115]]
Accurancy: 0.99800796812749
[[ 88   0   0   0   0]
 [  0  99   0   1   0]
 [  0   0 100   0   0]
 [  0   0   0 111   0]
 [  0   0   0   0 103]]
Accurancy: 0.99800796812749
[[ 87   0   0   0   0]
 [  0 102   0   0   0]
 [  0   0 109   0   0]
 [  0   0   0 106   0]
 [  0   0   0   0  98]]
Accurancy: 1.0
[[117   0   0   0   0]
 [  0 100   0   0   0]
 [  0   1 113   0   0]
 [  0   0   0  85   0]
 [  0   0   0   0  86]]
Accurancy: 0.99800796812749


In [143]:
pano_test_path = "D:\Pano_4096_2048\ZT\zt1knLZN3ZW2ROPCkMjyJg_4096_2048.jpg"

pano = np.array(Image.open(pano_test_path))

print(pano.shape)



(2048, 4096, 3)


In [146]:
test = pano[1:100, 1:100,:]

test.shape

(99, 99, 3)

In [ ]:
for x in range(0, )